# Introduction to Machine Learning - Final Project
### A Deep Reinforcement Algorithm in an OpenAI Gym Environment

During this final project, we will build a deep neural network and use reinforcement learning to solve a cart and pole balancing problem using OpenAI.  OpenAI Gym is a tookit for developing and comparing reinforcement learning algorithms that was built by OpenAI, a non-profit artificial intelligence research company founded by Elon Musk and Sam Altman.  

## 1. Installing and Loading Libraries

As always, the first step is to download and install all the dependencies.  To install OpenAI Gym, we will use Git. Depending on your operating system, the installation will be slightly different. Installation instructions can be found at:

https://git-scm.com/book/en/v2/Getting-Started-Installing-Git

Once Git is installed, we can download and install OpenAI easily using the following commands:


This downloads the bare minimums for the OpenAI Gym environment.  OpenAI Gym contains a wide variety of environments, even Atari games. Some of these environments will need additional dependencies installed.    

We will also need the following python libraries:

* NumPy
* Keras
* Theano
    
Install them using conda or pip.  

## 1.1 Importing libraries

Check that the libraries are installed correctly by running the following import statements

In [1]:
import gym
print(gym.__version__)

0.9.3


In [2]:
import keras
print(keras.__version__)

Using Theano backend.


2.0.5


In [3]:
import random
import math
import numpy as np
from collections import deque

## 1.2 OpenAI Gym Environment

The following cells build the 'CartPole-v0' environment that we will use in this project and runs it for twenty episodes.  

In [ ]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

To better understand the environment, lets take a look at the possible actions and observations for the 'CartPole-v0' environment.  

In [ ]:
print(env.action_space)
print(env.observation_space)

The Discrete() space allows a fixed range of non-negative numbers.  In this case, valid actions are either 0 or 1.  The Box space represents an n-dimensional box.  Valid observations for this case will be an array of four numbers. We can also check the Box's bounds:

In [ ]:
print(env.observation_space.high)
print(env.observation_space.low)

## 2. Define Parameters

In [4]:
# Training Parameters
n_episodes=1000
n_win_ticks=195
max_env_steps=None
gamma=1.0 # Discount factor. Consideration of future rewards - same policies (the ones that balance the pole) are optimal 
          # at every step.
epsilon=1.0 # Exploration. Agent chooses action that it believes has the best long-term effect with probability 1- epsilon,
            # and it chooses an action uniformly at random, otherwise.  
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01 # the learning rate, determines to what extent the newly acquired information will override the old information. 
alpha_decay=0.01
batch_size=64
monitor=False
quiet=False

# Environment Parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

[2017-12-14 15:15:23,964] Making new env: CartPole-v0


## 2.1 Building the Neural Network

The following cells define the model we will be using.

In [5]:
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

## 2.2 Defining Necessary Functions

It is best practice to define common operations as functions.  The following operations will be used often, so we define a function for each.

In [6]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

## 3. Defining Run Function

Lets define a function, run() that records the environment state and uses our network to choose the best action.  

In [7]:
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 100 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, epsilon)

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

## 4. Train the Network!

Here is the complete code together. Because Jupyter runs on a kernel, this code will work best if a single cell is used to run the entire code.

In [ ]:
import random
import gym
import math
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Training Parameters
n_episodes=1000
n_win_ticks=195
max_env_steps=None
gamma=1.0
epsilon=1.0
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01
batch_size=64
monitor=False
quiet=False

# Environment Parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay    
        
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 20 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, get_epsilon(e))

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

run()

[2017-12-14 15:16:27,203] Making new env: CartPole-v0


[Episode 0] - Mean survival time over last 100 episodes was 38.0 ticks.
[Episode 20] - Mean survival time over last 100 episodes was 16.380952381 ticks.
[Episode 40] - Mean survival time over last 100 episodes was 33.8536585366 ticks.
[Episode 60] - Mean survival time over last 100 episodes was 51.131147541 ticks.
[Episode 80] - Mean survival time over last 100 episodes was 54.7160493827 ticks.
